In [1]:
import requests
import os
from openai import AzureOpenAI
from docx import Document 

# Set up your Azure OpenAI API endpoint and key
# api_key = '7ac95627fe8f43bc9180f644b1294952'  # Replace with your actual API key
# endpoint = 'https://labnotes-instance.openai.azure.com'  # Replace with your endpoint URL

def summarize_with_azure_gpt(text):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {api_key}',
        'api-key': api_key
    }
    data = {
        "prompt": f"Summarize the following text:\n\n{text}",
        "max_tokens": 200,  # Adjust based on desired summary length
        "temperature": 0.5
    }
    response = requests.post(f"{endpoint}/openai/deployments/gpt-4/chat/completions", headers=headers, json=data)
    # response = requests.post(f"{endpoint}/openai/deployments/labnotes-deployment/completions", headers=headers, json=data)
    response_json = response.json()
    print("Res", response.text)
    summary = response_json['choices'][0]['text'].strip()
    return summary


def new_summarize_api(text):
    api_key = 'af24db8a03ec4d7e89f72df96ed8af05'  # Replace with your actual API key
    endpoint = 'https://labnotes-instance.openai.azure.com'  # Replace with your endpoint URL
        
    client = AzureOpenAI(
        api_key=api_key,  
        api_version="2024-07-01-preview",
        azure_endpoint=endpoint
    )
    
    chat_completion = client.chat.completions.create(
        # model="labnotes-deployment",
        model="gpt-4o",
        # prompt= "Hello",
        messages=[
            # {"role": "system", "content": "You are a helpful assistant."},
            # {"role": "user", "content": f"Generate a summary with the following sections: Aim, Materials, Procedure, Results, Conclusion. In addition to this, also include specific improvements for chemists:\n\n{text}"},  
            {"role": "user", "content": f"I have provided a set of experiments including procedure, results, conclusion in separate files. Can you provide a summary of the aggregate observations across experiments? In addition to this, also include specific chemistry improvements, new synthetic approaches that I can try, and any other bases that I can use etc.:\n\n{text}"},  
             # "Please summarize the following text...  I have a blue pen, and an other blue pen & a red pen."},
        ]
        # messages=[
        #     {"role": "system", "content": "You are a helpful assistant."},
        #     {"role": "user", "content": "Knock knock."},
        #     {"role": "assistant", "content": "Who's there?"},
        #     {"role": "user", "content": "Orange."},
        # ]
    )
    return chat_completion.choices[0].message.content

def read_docx(file):
    print(4, file.filename)
    if file.filename.endswith('.docx') and not file.filename.startswith('~$'):
        # file_path = os.path.join(subdir, file)
        # Read the .docx file
        print(31)
        doc = Document(file.file)
        print(3)

        combined_text = ""
        # combined_text += '**** Experiment '+str(file_counter) +': \n' 
        # Loop through each element (paragraph or table) in document order
        for element in doc.element.body:
            if element.tag.endswith('p'):  # Paragraph
                para = element
                # Use the Document object to get the paragraph text
                combined_text += get_paragraph_text(para) + "\n"
            elif element.tag.endswith('tbl'):  # Table
                table = element
                # Extract the table data directly from the table element
                combined_text += extract_table_data(table) + "\n"

    return combined_text

def read_docx_files_in_order(root_folder):
    # Initialize an empty string to store the combined text
    combined_text = ""
    file_counter = 0

    # Walk through all subfolders and files in the root folder
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            # Check if the file has a .docx extension
            if file.endswith('.docx') and not file.startswith('~$'):
                file_counter += 1
                file_path = os.path.join(subdir, file)
                # Read the .docx file
                doc = Document(file_path)

                combined_text += '**** Experiment '+str(file_counter) +': \n' 
                # Loop through each element (paragraph or table) in document order
                for element in doc.element.body:
                    if element.tag.endswith('p'):  # Paragraph
                        para = element
                        # Use the Document object to get the paragraph text
                        combined_text += get_paragraph_text(para) + "\n"
                    elif element.tag.endswith('tbl'):  # Table
                        table = element
                        # Extract the table data directly from the table element
                        combined_text += extract_table_data(table) + "\n"

    return combined_text

def get_paragraph_text(paragraph_element):
    paragraph_text = ""
    for child in paragraph_element.iter():
        if child.tag.endswith('t'):  # Text element within a paragraph
            paragraph_text += child.text if child.text else ''
    return paragraph_text

def extract_table_data(table_element):
    table_data = ""
    for row in table_element.xpath('.//w:tr'):
        row_data = []
        for cell in row.xpath('.//w:tc'):
            cell_text = "".join(cell.xpath('.//w:t/text()'))
            row_data.append(cell_text)
        table_data += "\t".join(row_data) + "\n"
    return table_data

def main():
    # Read text from a file
    # with open('data/text-files/1-010.txt', 'r', encoding='utf-8') as file:
    # text = file.read()

    all_experiments = read_docx_files_in_order('data/word_data')

    # print("Ass", all_experiments)

    # Summarize the text
    # summary = summarize_with_azure_gpt(text)
    summary = new_summarize_api(all_experiments)

    # Print and save the summary
    print("Summary:")
    print(summary)

    with open("summary002.txt", "w", encoding="utf-8") as file:
        file.write(summary)

if __name__ == "__main__":
    main()


Summary:
Sure, I'd be happy to help with that. Please provide the content of the files for a comprehensive summary and further recommendations regarding chemistry improvements, new synthetic approaches, and possible bases you could use. Once I have the details, I can process the information and provide a detailed summary and suggestions.
